In [12]:
import pandas as pd
import re
import os
from pandas import json_normalize
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import pandas as pd
import os
from pandas import json_normalize
import numpy as np
import matplotlib.pyplot as plt

directory_path = 'All_Json_2023-2024'

dataframes = []

def extract_message(text_entities):
    try:
        # Concatenate the 'text' fields from each dictionary in the list
        return ''.join([entity['text'] for entity in text_entities if 'text' in entity])
    except:
        return None

# Iterate through every file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.json'):
        file_path = os.path.join(directory_path, filename)
        try:
            df = pd.read_json(file_path)

            # Ensure 'messages' column exists
            if 'messages' in df.columns:
                df = json_normalize(df['messages'])
            else:
                print(f"Skipping {filename} as it doesn't have 'messages' field.")
                continue
            
            # Check if the necessary columns are available
            if 'date' in df.columns and 'text_entities' in df.columns:
                # Clean the data
                df = df[['date', 'text_entities']]

                # Filter rows where 'text_entities' is a non-empty list
                df = df[df['text_entities'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

                # Extract the message text
                df['message'] = df['text_entities'].apply(extract_message)

                # Drop 'text_entities' column
                df_cleaned = df.drop(columns=['text_entities'])
                df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
                df_cleaned = df_cleaned.sort_values(by='date', ascending=False)

                # Add a new column for the channel name (based on filename)
                df_cleaned['channel_name'] = filename[:-5]  # removes the '.json' from the filename

                # Append the dataframe to the list
                dataframes.append(df_cleaned)
            else:
                print(f"Skipping {filename} as it doesn't contain required columns: 'date' and 'text_entities'.")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Combine all dataframes into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Remove duplicates based on the 'message' column
final_df = combined_df.drop_duplicates(subset=['message'])

# Save to CSV
final_df.to_csv('all_2024.csv', index=False)

print(final_df.shape)
print(final_df.head())


Skipping sdam_snimu_astana.json as it doesn't contain required columns: 'date' and 'text_entities'.
Skipping Подселение Шымкент найти комнату квартиру.json as it doesn't contain required columns: 'date' and 'text_entities'.
(37545, 3)
                 date                                            message  \
0 2024-11-27 14:55:46  Всем привет , срочно ищу 3 девушку для прожива...   
1 2024-11-25 20:38:21  62500 + комм услуги, 25000 за депозит\nБукар Ж...   
2 2024-11-25 20:37:46  всем привет,ищу девушку для сожительства,будем...   
3 2024-11-25 20:37:35  Всем привет\nИщем к кому (к девушкам) подселит...   
4 2024-11-25 20:37:24  Ищем квартиру\nМы, три девушки, ищем квартиру ...   

  channel_name  
0   DAILY ХАТА  
1   DAILY ХАТА  
2   DAILY ХАТА  
3   DAILY ХАТА  
4   DAILY ХАТА  


In [10]:
final_df = final_df.dropna(how='any')
final_df.head()
final_df.shape

(37545, 3)

In [15]:
def clean_text(text):
    cleaned_text = re.sub(r'[^А-Яа-яӘәІіҢңҒғҚқҰұҮүҺһӨөЁё0-9\s]', '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

final_df['message'] = final_df['message'].apply(clean_text)
final_df = final_df[final_df['message'] != ""]
final_df = final_df[~final_df['message'].str.startswith("Извините")]

final_df = final_df[final_df['message'].apply(lambda x: len(x.split()) >= 10)]
final_df

,date,message,channel_name
0,2024-11-27 14:55:46,Всем привет срочно ищу 3 девушку для проживани...,DAILY ХАТА
1,2024-11-25 20:38:21,62500 комм услуги 25000 за депозит Букар Жырау...,DAILY ХАТА
2,2024-11-25 20:37:46,всем приветищу девушку для сожительствабудем ж...,DAILY ХАТА
3,2024-11-25 20:37:35,Всем привет Ищем к кому к девушкам подселиться...,DAILY ХАТА
4,2024-11-25 20:37:24,Ищем квартиру Мы три девушки ищем квартиру для...,DAILY ХАТА
...,...,...,...
58205,2024-02-11 00:07:21,Квартира помесячно мкр 5 ул Черненко 5 Уральск...,Уральск Квартира Орал КВАРТИРА Аренда
58207,2024-02-10 23:59:15,Квартира на длительный срок мкр Кунаева 55 Абу...,Уральск Квартира Орал КВАРТИРА Аренда
58208,2024-02-10 23:54:12,Квартира на длительный срок Маметова проспект ...,Уральск Квартира Орал КВАРТИРА Аренда
58209,2024-02-10 23:52:18,Квартира на длительный срок мкр СевероВосток 2...,Уральск Квартира Орал КВАРТИРА Аренда


In [16]:
final_df.to_csv('all_2024.csv', index=False)


In [17]:
final_df.shape

(32660, 3)